In [1]:
import os
from openai import OpenAI

client = OpenAI()

# API key
# print(os.environ["OPENAI_API_KEY"])

In [61]:
# List all files
!curl https://api.openai.com/v1/files -H "Authorization: Bearer $OPENAI_API_KEY"

{
  "object": "list",
  "data": [
    {
      "object": "file",
      "id": "file-HynYH547MTmZQR7nG71sZf",
      "purpose": "fine-tune",
      "filename": "val_300.jsonl",
      "bytes": 130502,
      "created_at": 1743319240,
      "expires_at": null,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-VdNNYHJr2Si61iwvvwWQmb",
      "purpose": "fine-tune",
      "filename": "train_3000.jsonl",
      "bytes": 1310709,
      "created_at": 1743319236,
      "expires_at": null,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-TWXePzQT91jWxA4QgWPLP4",
      "purpose": "fine-tune",
      "filename": "val_200.jsonl",
      "bytes": 86728,
      "created_at": 1743270368,
      "expires_at": null,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-GBQXTRJdEjHB9MzUqxtNUz",
      "purpose": "fine-tune",
      "f

In [3]:
# List all available models
models = client.models.list()
for model in models:
    if "system" in model.owned_by.lower():
        print(f"ID: {model.id}")
        print(f"Owned by: {model.owned_by}")
        print("-" * 50)

ID: gpt-4o-realtime-preview-2024-12-17
Owned by: system
--------------------------------------------------
ID: gpt-4o-audio-preview-2024-12-17
Owned by: system
--------------------------------------------------
ID: dall-e-3
Owned by: system
--------------------------------------------------
ID: dall-e-2
Owned by: system
--------------------------------------------------
ID: gpt-4o-audio-preview-2024-10-01
Owned by: system
--------------------------------------------------
ID: gpt-4o-mini-realtime-preview-2024-12-17
Owned by: system
--------------------------------------------------
ID: gpt-4o-mini-realtime-preview
Owned by: system
--------------------------------------------------
ID: gpt-4o-realtime-preview-2024-10-01
Owned by: system
--------------------------------------------------
ID: gpt-4o-transcribe
Owned by: system
--------------------------------------------------
ID: gpt-4o-mini-transcribe
Owned by: system
--------------------------------------------------
ID: gpt-4o-realtim

In [ ]:
# Delete specific models

models_to_delete = [
    "ft:gpt-4o-mini-2024-07-18:personal::BGUCkgZ0:ckpt-step-500",
    "ft:gpt-4o-mini-2024-07-18:personal::BGUCkQiR:ckpt-step-1000",
    "ft:gpt-4o-mini-2024-07-18:personal:ft-3000:BGis5Maf:ckpt-step-1500",
    "ft:gpt-4o-mini-2024-07-18:personal:ft-3000:BGis6EPc:ckpt-step-3000",
    "ft:gpt-4o-mini-2024-07-18:personal:ft-2000:BGVhkJ4W:ckpt-step-1000",
    "ft:gpt-4o-mini-2024-07-18:personal:ft-2000:BGVhklGi:ckpt-step-2000",
    "ft:gpt-4o-mini-2024-07-18:personal:202503291345-emo-all-auto:BGKSG4Oa:ckpt-step-100",
    "ft:gpt-4o-mini-2024-07-18:personal:202503291345-emo-all-auto:BGKSGv2l:ckpt-step-200",
    "ft:gpt-4o-mini-2024-07-18:personal:202503291345-emo-all-auto:BGKSHA1k"
]

for model in models_to_delete:
    try:
        response = client.models.delete(model)
        print(f"Successfully deleted model: {response.id}")
    except Exception as e:
        # print(f"Error deleting model: {e}")
        continue

In [5]:
client.fine_tuning.jobs.create(
    training_file="file-VdNNYHJr2Si61iwvvwWQmb",
    validation_file="file-HynYH547MTmZQR7nG71sZf",
    model="gpt-4o-mini-2024-07-18",
    method={
        "type": "supervised",
        "supervised": {
            "hyperparameters": {
                "n_epochs": 5, 
                "batch_size": 4, 
                "learning_rate_multiplier": 0.01
            },
        },
    },
    integrations=[{
        "type": "wandb",
        "wandb": {
            "project": "nlp-ft",
            "name": "ft-3000-ep5-bs4-lr001",
        }
    }],
    # seed=908,
    suffix="ft-3000"
)

FineTuningJob(id='ftjob-3Lfbi1lPVFO1G2hjqwNpxXRR', created_at=1743646994, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=4, learning_rate_multiplier=0.01, n_epochs=5), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-RdrRilXIivyn4jVe1k2lbkRE', result_files=[], seed=1072340086, status='validating_files', trained_tokens=None, training_file='file-VdNNYHJr2Si61iwvvwWQmb', validation_file='file-HynYH547MTmZQR7nG71sZf', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='nlp-ft', entity=None, name=None, tags=None, run_id='ftjob-3Lfbi1lPVFO1G2hjqwNpxXRR'))], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=4, learning_rate_multiplier=0.01, n_epochs=5)), type='supervised'), user_provided_suffix='ft-3000')

In [74]:
client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-9DYJyI2jgfWoKivHpE4c6At8', created_at=1743437274, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=2, learning_rate_multiplier=0.001, n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-RdrRilXIivyn4jVe1k2lbkRE', result_files=[], seed=908, status='running', trained_tokens=None, training_file='file-VdNNYHJr2Si61iwvvwWQmb', validation_file='file-HynYH547MTmZQR7nG71sZf', estimated_finish=1743438855, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='nlp-ft', entity=None, name=None, tags=None, run_id='ftjob-9DYJyI2jgfWoKivHpE4c6At8'))], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=2, learning_rate_multiplier=0.001, n_epochs=1)), type='supervised'), user_provided_suffix='e